In [90]:
using LMCLUS
include("NMI.jl")
using FreqTables

In [79]:
data_original = readdlm("d:\\LMC_Datasets\\data_3_4_5_30.csv", header=false, ',');
trueLab = data_original[end,:];
data = data_original[1:end-1,:];

#color map for plots
cDict = Dict("1" => "red", "2" => "blue", "3" => "green", "4" => "violet", "5"=>"black", "6"=>"orange", "7"=>"magenta");

In [80]:
function normalizeArr(x)
    for i=1:size(x)[1]
        x[i,:] = (x[i,:]-minimum(x[i,:]))/(maximum(x[i,:])-minimum(x[i,:]));
    end
end

normalizeArr(data);

In [81]:
function genPredLab(Ms, l)
    predLab = ones(l);

    for i=1:length(Ms)
        lab = labels(Ms[i]);
        predLab[lab] = i;
    end

    return predLab;
end

genPredLab (generic function with 1 method)

In [82]:
params = LMCLUS.Parameters(2);
Ms = lmclus(data,params)

5-element Array{LMCLUS.Manifold,1}:
 Manifold (dim = 1, size = 917) 
 Manifold (dim = 1, size = 1024)
 Manifold (dim = 1, size = 988) 
 Manifold (dim = 1, size = 1002)
 Manifold (dim = 0, size = 69)  

In [84]:
predLab = genPredLab(Ms, length(trueLab));
print(getNMI(freqtable(trueLab,predLab)))
freqtable(trueLab,predLab)

0.95

4×5 Named Array{Int64,2}
Dim1 ╲ Dim2 │  1.0   2.0   3.0   4.0   5.0
────────────┼─────────────────────────────
1.0         │    0   998     0     0     2
2.0         │    0    10   988     2     0
3.0         │  917    16     0     0    67
4.0         │    0     0     0  1000     0

In [89]:
trace1 = scatter3d(;x=data_original[1,:], y=data_original[2,:], z=data_original[3,:], mode="markers", marker_size=0.5, marker_color=map(x->cDict[string(Int(x))], trueLab))
layout = Layout(;title="Original Clusters");
plot(trace1,layout)
#trace2 = scatter3d(;x=data_original[1,1:2], y=[minimum(data_original[2,:]),maximum(data_original[2,:])], z=[minimum(data_original[3,:]),maximum(data_original[3,:])], mode="lines")

In [74]:
trace1 = scatter3d(;x=data_original[1,:], y=data_original[2,:], z=data_original[3,:], mode="markers", marker_size=0.5, marker_color=map(x->cDict[string(Int(x))], predLab))
C1 = mean(data_original[1:3,Ms[1].points],2);
trace2 = scatter3d(;x=data_original[1,1:2], y=[minimum(data_original[2,:]),maximum(data_original[2,:])], z=[minimum(data_original[3,:]),maximum(data_original[3,:])], mode="lines")
layout = Layout(;title="LMCLUS Clusters");

In [109]:
mu1

3-element Array{Float64,1}:
 0.206427 
 0.0719201
 0.275161 

In [120]:
#Plot the clustered points
trace1 = scatter3d(;x=data_original[1,:], y=data_original[2,:], z=data_original[3,:], mode="markers", marker_size=0.5, marker_color=map(x->cDict[string(Int(x))], predLab))

#Plot the first clusters manifold
C1 = mean(data_original[1:3,Ms[1].points],2);
mu1 = Ms[1].μ;
trace2 = scatter3d(;x=[C1[1]-20*mu1[1],C1[1]+20*mu1[1]], y=[C1[2]-20*mu1[2],C1[2]+20*mu1[2]], z=[C1[3]-20*mu1[3],C1[3]+20*mu1[3]], mode="lines", marker_color="red")

#Plot the second clusters manifold
C2 = mean(data_original[1:3,Ms[2].points],2);
mu2 = Ms[2].μ;
trace3 = scatter3d(;x=[C2[1]-10*mu2[1],C2[1]+10*mu2[1]], y=[C2[2]-10*mu2[2],C2[2]+10*mu2[2]], z=[C2[3]-10*mu2[3],C2[3]+10*mu2[3]], mode="lines", marker_color="blue")

#Plot the third clusters manifold
C3 = mean(data_original[1:3,Ms[3].points],2);
mu3 = Ms[3].μ;
trace4 = scatter3d(;x=[C3[1]-10*mu3[1],C3[1]+10*mu3[1]], y=[C3[2]-10*mu3[2],C3[2]+10*mu3[2]], z=[C3[3]-10*mu3[3],C3[3]+10*mu3[3]], mode="lines", marker_color="green")

#Plot the fourth clusters manifold
C4 = mean(data_original[1:3,Ms[4].points],2);
mu4 = Ms[4].μ;
trace5 = scatter3d(;x=[C4[1]-5*mu4[1],C4[1]+5*mu4[1]], y=[C4[2]-5*mu4[2],C4[2]+5*mu4[2]], z=[C4[3]-5*mu4[3],C4[3]+5*mu4[3]], mode="lines", marker_color="green")

layout = Layout(title="LMCLUS Clusters",showlegend=false);
plot([trace1,trace2,trace3,trace4,trace5],layout)

In [126]:
function plotHist(M, data, predLab)
    Dis = distance_to_manifold(data,M.μ, M.proj);
    index = 1:size(data,2);
    b = bar(;x=index, y=Dis, marker_color=map(x->cDict[string(Int(x))], predLab), line=false);
    return(b)
end

plotHist (generic function with 1 method)

In [130]:
layout = Layout(title="Distance histogram",showlegend=false,xaxis_ticks=false);
b1 = plotHist(Ms[1],data, predLab);
index = setdiff(1:size(data,2),Ms[1].points);
b2 = plotHist(Ms[2], data[:,index], predLab[index]);
index = setdiff(index,Ms[2].points);
b3 = plotHist(Ms[3], data[:,index], predLab[index]);
index = setdiff(index,Ms[3].points);
b4 = plotHist(Ms[4], data[:, index], predLab[index]);

p = [plot(b1,layout) plot(b2,layout); plot(b3,layout) plot(b4,layout)];
p.plot.layout["showlegend"] = false;
p